# Setting NTP Servers for HPE iLO Systems

<div class="alert  alert-block  alert-info" style="border-radius: 20px;">    
    <ul>
        <li><p>This Jupyter Notebook demonstrates an example of collecting ESKM (Enterprise Secure Key Manager) data for HPE iLO (Integrated Lights-Out) systems. It utilizes the HPE iLOrest library to interact with the iLO API and retrieve the necessary information.</p></li>
        <li>As you are here let's get it done easily.</li>
        <li>To perform this you need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style="none">python-ilorest-library</a> installed in your machine.</li>
        <li>Please keep your username and password handy</li>
        <li>Do execute each successive cell once to avoid any errors!</li>
    </ul>    
</div>

<div class="alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary python libraries</h3>
    <ul>
        <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys</a></li>
        <li>json</li>
        <li>redfish</li> 
    </ul>
</div>

In [1]:
import sys
import json
import time
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

from get_resource_directory import get_resource_directory
from reset_ilo import reset_ilo
from enable_ntp_servers import enable_ntp

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <div style = "color:green; font-size:40px;">2.</div><br>
    <p>The script begins by defining the necessary dependencies and helper functions. The set_ilo_ntp_servers function is responsible for updating the NTP server settings for the iLO system.</p>
<br>Steps involved are:<br>

1. The function set_ilo_ntp_servers sets the NTP (Network Time Protocol) servers for an HPE iLO system. It first retrieves the URI for the date and time service using the get_resource_directory function. If the resource directory is disabled or not available, it finds the URI by traversing the managers' hierarchy.

2. Once the URI is obtained, the function sends a GET request to retrieve the current NTP server settings. If the StaticNTPServers property is present, it sends a PATCH request to update the NTP server list with the provided ntp_server_list.

3. If any error occurs during the process, the function handles it accordingly. If the response status is 400, it attempts to access the iLO extended error information and prints it. If the response status is not 200 or 400, it prints an error message.

4. If the NTP server update is successful, it prints a success message and the updated NTP server information.

<br><br>
In summary, the function retrieves the URI for the date and time service, updates the NTP server settings, and handles any errors that may occur during the process.
</div>

In [1]:
def set_ilo_ntp_servers(_redfishobj, ntp_server_list):

    date_time_uri = None

    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
        date_time_uri = managers_members_response.obj.Oem.Hpe.Links['DateTimeService']['@odata.id']
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#HpeiLODateTime.' in instance['@odata.type']:
                date_time_uri = instance['@odata.id']

    if date_time_uri:
        data = _redfishobj.get(date_time_uri)
        if data.dict.get('StaticNTPServers'):
            resp = _redfishobj.patch(date_time_uri, {'StaticNTPServers': ntp_server_list})
        else:
            raise Exception("\'StaticNTPServers\' property is not available/modifyable.\n")
        #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
        #error message to see what went wrong
        if resp.status == 400:
            try:
                print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                                                                sort_keys=True))
            except Exception as excp:
                sys.stderr.write("A response error occurred, unable to access iLO Extended " \
                                 "Message Info...")
        elif resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("Success!\n")
            print(json.dumps(resp.dict, indent=4, sort_keys=True))
            ntp_data = _redfishobj.get(date_time_uri).dict.get('StaticNTPServers')
            print("Printing updated NTP Servers:\n")
            print(json.dumps(ntp_data, indent=4, sort_keys=True))

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <div style = "color:green; font-size:40px;">3.</div><br>
    The following code defines the give_client function, which creates a client object and performs the login operation through the following steps:<br>
1. The function give_client creates a Redfish client object by instantiating the RedfishClient class. It provides the base URL, username, and password for the Redfish service as parameters.

2. The function attempts to log in to the Redfish service using the created client object. It sends the login request with the provided credentials.

3. If the login is successful, the function returns the instantiated Redfish client object. If there is an error connecting to the server or if the server does not support Redfish, an error message is printed, and the program exits.
<br><br>
In summary, the give_client function creates a Redfish client object, performs the login process, and returns the client object if the login is successful.</div>

In [ ]:
def give_client():
    try:
        # Create a Redfish client object
        rf_obj = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        rf_obj.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    return rf_obj

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
<div style = "color:green; font-size:40px;">4.</div><br>
    This is Main function overview.<br>
    <i>Let me give you a walk through of it!</i>
    <br><br>
    The main section of the script sets the system URL, login account, and password. It creates a Redfish client object, logs in to the iLO system, enables NTP servers for the primary iLO management interface, resets the iLO, and waits for 60 seconds. Afterward, it logs out, creates a new Redfish client object, and calls the set_ilo_ntp_servers function to configure the NTP servers using the specified list. Finally, it logs out again.

 </div>

<div class="alert alert-block alert-warning" style="border-radius: 20px;">
<b> When running on the server locally : </b>  use the following values <br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    LOGFILE = data.ahs<br>
    <b> When running remotely </b> connect using the secured (https://) address,account name, and password to send https requests <br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

In [ ]:
if __name__ == "__main__":
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Name: ")
    LOGIN_PASSWORD = input("Enter password: ")

    # logfile path and filename
    LOGFILE = input("Enter log file path and file name: ")
    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    get_ahs_data(REDFISHOBJ, LOGFILE)
    REDFISHOBJ.logout()

<div class = "alert alert-block alert-success" style="border-radius: 20px;">In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>